# Llamafile x Autogen

I want to test out using an opensource LLM like mistral 7B or Llama 2 locally using llamafile. 

[http://127.0.0.1:8080](http://127.0.0.1:8080)

In [17]:
import autogen 
from autogen import AssistantAgent, UserProxyAgent

config_list=[
    {
        "base_url": "http://127.0.0.1:8080/v1",
        "api_key": "NULL", # just a placeholder
    }
]

llm_config_mistral={
    "config_list": config_list,
}


In [18]:
llm_config_mistral

{'config_list': [{'base_url': 'http://127.0.0.1:8080/v1', 'api_key': 'NULL'}]}

In [21]:
coding_assistant = AssistantAgent(
    name="coding_assistant",
    llm_config=llm_config_mistral
)

coding_runner = UserProxyAgent(
    name="coding_runner",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg = lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    llm_config=llm_config_mistral,
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False
    },
)

In [23]:
user_message = "Write a fibonacci sequence function in python and run it to print 20 numbers. Say TERMINATE when the task is finished and you got the expected result"
coding_runner.initiate_chat(coding_assistant, 
                            message=user_message)

coding_runner (to coding_assistant):

Write a fibonacci sequence function in python and run it to print 20 numbers. Say TERMINATE when the task is finished and you got the expected result

--------------------------------------------------------------------------------
coding_assistant (to coding_runner):

Sure, here's a simple Python function that generates a Fibonacci sequence. It takes the number of terms as an argument.

```python
def fibonacci(n):
    fib_sequence = [0, 1]
    while len(fib_sequence) < n:
        fib_sequence.append(fib_sequence[-1] + fib_sequence[-2])
    return fib_sequence
```

Now, let's print the first 20 numbers in the Fibonacci sequence:

```python
print(fibonacci(20))
```

When you run this code, it should print the first 20 numbers in the Fibonacci sequence. Please let me know if you need further assistance.

Please note that the Fibonacci sequence is a series of numbers in which each number is the sum of the two preceding ones, usually starting with 0 an